# Qwen-VL 端到端微调与优化

这份教程的目标，是带大家走完一遍完整的流程，一步步地在自己的数据集上，把 Qwen-VL 模型调好。在这个过程中，我们不只是跑代码，更重要的是搞清楚每一步是在做什么，为什么要这么做。最终的目标是，每个人都能掌握一套可以重复使用、标准化的 VLM 微调工作流。

## 引言：我们的目标


1. **环境搭建**：配置一个干净、版本兼容的、可用于 VLM 微调的工作环境。
2. **数据处理 (YOLOv8 范式)**：从原始数据源出发，将其转换为 Qwen-VL 微调所需的、包含图像和结构化对话的特定格式，并借鉴 YOLOv8 的思路进行严格的数据校验。
3. **模型微调 (SFT & LoRA)**：使用 LLaMA-Factory 或 `trl` 库，在我们的自定义数据上对 Qwen-VL 模型进行高效的 LoRA SFT 微调。
4. **结果分析与推理**：学习如何解读训练过程中的损失曲线，并使用微调后的模型对新图片进行预测，验证微调效果。

现在，让我们开始吧！

## 第一部分：环境与项目设置

一个良好规划的项目结构是成功的一半。我们将首先创建必要的目录，并安装所需的Python库。

### 1.1 安装依赖库
Qwen-VL 的微调依赖于 transformers, peft, trl 等库。一个稳定且版本兼容的开发环境至关重要。

In [ ]:
# 核心库安装 (请根据您的 CUDA 版本选择合适的 PyTorch 安装命令)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Hugging Face 生态系统和微调工具
!pip install "transformers>=4.41.0,<5.0.0"
!pip install -q "accelerate>=0.30.1"
!pip install -q "peft>=0.10.0"
!pip install -q "datasets>=2.19.1"
!pip install -q "trl"

# Qwen-VL 特定库和性能优化库
!pip install -q "bitsandbytes>=0.43.1"
!pip install -q "qwen-vl-utils>=0.0.11"
# pip install -q "flash-attn==2.5.8" --no-build-isolation # 可选，根据GPU支持情况安装

# 可视化和监控工具
!pip install -q "supervision>=0.20.0"
!pip install -q "tensorboard"
!pip install -q "wandb"

### 1.2 设置工作目录

我们将所有文件（数据、代码、结果）都存放在一个统一的根目录下，这有助于保持项目整洁，并方便路径管理。

In [ ]:
import os
import shutil # 用于文件操作

# 尝试处理 Google Colab 环境
try:
    from google.colab import drive
    print("--> 正在挂载 Google Drive...")
    drive.mount('/content/drive')
    print("Google Drive 挂载成功！")
    # 在 Google Drive 中定义项目根目录以实现持久化存储
    BASE_DIR = "/content/drive/MyDrive/Qwen_VL_Finetune_Project"
except ImportError:
    # 在本地环境中，将项目根目录设置在当前工作目录下
    BASE_DIR = "./Qwen_VL_Finetune_Project"
    print(f"非 Colab 环境，项目根目录设置为: {os.path.abspath(BASE_DIR)}")

# 如果旧目录存在，则清理，确保从干净的状态开始
if os.path.exists(BASE_DIR):
    print(f"发现旧的项目目录 '{BASE_DIR}'，正在清理...")
    shutil.rmtree(BASE_DIR)

# 创建项目根目录
os.makedirs(BASE_DIR, exist_ok=True)
print(f"项目根目录已创建: {BASE_DIR}")

# 切换当前工作目录至项目根目录，这是非常关键的一步！
os.chdir(BASE_DIR)
print(f"当前工作目录已切换至: {os.getcwd()}")

## 第二部分：数据准备

这是整个流程中最核心的一步。将原始数据集转换为 Qwen-VL 可以“理解”的指令对话格式，并进行验证。

### 2.1 定义全局配置

为了方便管理和修改实验参数，我们将所有重要的配置项都定义在一个地方。

In [ ]:
from dataclasses import dataclass, field

@dataclass
class DataConfig:
    """管理数据处理的所有配置"""
    # Hugging Face Hub 上的数据集名称
    DATASET_NAME: str = "detection-datasets/coco"
    DATASET_SPLIT: str = "train"

    # 输出目录配置
    OUTPUT_DIR: str = "qwen_vl_dataset"
    IMAGE_DIR: str = field(init=False)
    ANNOTATION_FILE: str = field(init=False)

    # 为了快速演示，我们只处理部分样本
    NUM_SAMPLES_TO_PROCESS: int = 5000

    def __post_init__(self):
        self.IMAGE_DIR = os.path.join(self.OUTPUT_DIR, "images")
        self.ANNOTATION_FILE = os.path.join(self.OUTPUT_DIR, "annotations.jsonl")

# 实例化配置
data_cfg = DataConfig()

# 创建所有必需的目录
os.makedirs(data_cfg.IMAGE_DIR, exist_ok=True)
print(f"数据输出目录结构创建成功！图片将存放于: {data_cfg.IMAGE_DIR}")

### 2.2 格式转换：从原始标注到指令对话

这是最关键的转换逻辑。我们需要将每张图片和它的标注信息转换为 Qwen-VL SFT 所需的对话格式。每个 JSONL 行代表一个样本，包含图片路径和多轮对话。

In [ ]:
import json
from datasets import load_dataset
from PIL import Image
from tqdm import tqdm

def convert_sample_to_qwen_vl_format(
    sample, image_relative_path, id_to_name, normalize=True
):
    """
    将单个样本转换为 Qwen-VL SFT 对话格式。
    - normalize=True 时，输出 [0,1] 归一化坐标；否则输出像素坐标。
    - 坐标语义为 [x1,y1,x2,y2]，且 x1<x2, y1<y2。
    """
    img_w, img_h = sample["image"].size

    # 1) 更强约束的提示词（可按需要保留/删除 <image>）
    human_prompt = (
        "<image>\n"
        "请找出图中所有目标，并严格以 JSON 数组输出，每个元素为："
        "{\"bbox_2d\":[x1,y1,x2,y2],\"label\":\"类别名\"}。"
        f"{'坐标为归一化到[0,1]' if normalize else '坐标为原图像素'}，"
        "不要输出任何额外文字。"
    )

    # 2) 组装答案
    assistant_objects = []
    if "objects" in sample and sample["objects"].get("bbox"):
        for i, (x, y, w, h) in enumerate(sample["objects"]["bbox"]):
            cat_id = sample["objects"]["category"][i]
            label_text = id_to_name.get(cat_id, str(cat_id))

            x1, y1 = float(x), float(y)
            x2, y2 = float(x + w), float(y + h)

            # 裁剪到图像边界
            x1 = max(0.0, min(x1, img_w))
            y1 = max(0.0, min(y1, img_h))
            x2 = max(0.0, min(x2, img_w))
            y2 = max(0.0, min(y2, img_h))
            if x2 <= x1 or y2 <= y1:
                continue

            if normalize:
                bx1, by1 = x1 / img_w, y1 / img_h
                bx2, by2 = x2 / img_w, y2 / img_h
            else:
                bx1, by1, bx2, by2 = int(round(x1)), int(round(y1)), int(round(x2)), int(round(y2))

            assistant_objects.append({"bbox_2d": [bx1, by1, bx2, by2], "label": label_text})

    if not assistant_objects:
        # 也可选择保留负样本：assistant_objects = []
        return None

    return {
        "image": image_relative_path,
        "conversations": [
            {"from": "human", "value": human_prompt},
            {"from": "gpt", "value": json.dumps(assistant_objects, ensure_ascii=False, separators=(",", ":"))},
        ],
    }


# --- 执行数据处理 ---
print(f"正在从 Hugging Face 加载数据集: {data_cfg.DATASET_NAME}...")
# 使用 streaming=True 避免一次性将整个数据集加载到内存
full_dataset = load_dataset(data_cfg.DATASET_NAME, split=data_cfg.DATASET_SPLIT, streaming=True)
class_names = load_dataset(data_cfg.DATASET_NAME, split=data_cfg.DATASET_SPLIT).features['objects']['category'].feature.names

print(f"开始处理数据，将生成 {data_cfg.NUM_SAMPLES_TO_PROCESS} 条训练样本...")
with open(data_cfg.ANNOTATION_FILE, 'w', encoding='utf-8') as f:
    processed_count = 0
    for sample in tqdm(full_dataset, total=data_cfg.NUM_SAMPLES_TO_PROCESS, desc="处理样本"):
        if processed_count >= data_cfg.NUM_SAMPLES_TO_PROCESS:
            break

        image = sample['image']
        image_filename = f"img_{sample['image_id']}.jpg"
        image_save_path = os.path.join(data_cfg.IMAGE_DIR, image_filename)

        if not os.path.exists(image_save_path):
            if image.mode != 'RGB':
                image = image.convert('RGB')
            image.save(image_save_path)

        conversation = convert_sample_to_qwen_vl_format(sample, image_save_path, class_names)

        if conversation:
            f.write(json.dumps(conversation, ensure_ascii=False) + '\n')
            processed_count += 1

print(f"\n数据处理完成！总共生成了 {processed_count} 条训练数据。")
print(f"标注文件保存在: {data_cfg.ANNOTATION_FILE}")


### 2.3 数据验证：可视化抽查

“Garbage in, garbage out.” 在投入计算资源进行训练之前，验证数据的正确性至关重要。我们将随机抽取几条数据，将其可视化，直观地判断转换是否正确。

In [ ]:
import random
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

def visualize_qwen_vl_sample(jsonl_line):
    """可视化单个Qwen-VL样本，确认转换是否正确。"""
    try:
        data = json.loads(jsonl_line)
        image_path = data.get("image")
        if not image_path or not os.path.exists(image_path):
            print(f"错误：图像文件 '{image_path}' 不存在。")
            return None

        image = Image.open(image_path).convert("RGB")
        draw = ImageDraw.Draw(image)
        
        gpt_response_str = data["conversations"][1]["value"]
        detections = json.loads(gpt_response_str)

        colors = ["red", "green", "blue", "yellow", "purple", "orange"]
        
        print(f"在图像 '{os.path.basename(image_path)}' 上找到 {len(detections)} 个物体，正在绘制...")
        for i, det in enumerate(detections):
            box = det['bbox_2d']
            label = det['label']
            color = colors[i % len(colors)]
            
            draw.rectangle(box, outline=color, width=3)
            
            try:
                font = ImageFont.truetype("arial.ttf", 15)
            except IOError:
                font = ImageFont.load_default()
            
            text_bbox = draw.textbbox((box[0], box[1] - 15), label, font=font)
            draw.rectangle(text_bbox, fill=color)
            draw.text((box[0], box[1] - 15), label, fill="black", font=font)
            
        return image
    except Exception as e:
        print(f"可视化时发生错误: {e}")
        return None

# --- 执行可视化验证 ---
print("\n正在随机抽查1条数据进行可视化验证...")
with open(data_cfg.ANNOTATION_FILE, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    if not lines:
        print("标注文件为空，无法验证。")
    else:
        random_line = random.choice(lines)
        visualized_image = visualize_qwen_vl_sample(random_line)
        if visualized_image:
            plt.figure(figsize=(10, 10))
            plt.imshow(visualized_image)
            plt.axis('off')
            plt.show()

## 第三部分：监督微调 (SFT)

数据准备就绪，现在可以开始训练了！我们将使用 Hugging Face trl 库中的 SFTTrainer 进行第一阶段的微调。

### 3.1 方法一：使用 Hugging Face TRL 进行微调
trl 库是 Hugging Face 官方推荐的微调工具库，它与 transformers 和 peft 无缝集成，通过 SFTTrainer 可以用非常简洁的代码实现微调。

#### 3.1.1 定义 SFT 训练配置

我们将所有的训练超参数集中在一个配置类中，方便管理。

In [ ]:
from dataclasses import dataclass, field

@dataclass
class SFTConfig:
    # 模型与数据路径
    model_id: str = "Qwen/Qwen2.5-VL-7B-Instruct"
    data_path: str = data_cfg.ANNOTATION_FILE
    output_dir: str = "./qwen2.5-vl-sft-lora"

    # 训练循环控制
    num_train_epochs: int = 1
    per_device_train_batch_size: int = 2 # 根据显存调整
    gradient_accumulation_steps: int = 8 # 有效批大小 = 2 * 8 = 16
    gradient_checkpointing: bool = True

    # 优化器与学习率
    learning_rate: float = 2e-5
    weight_decay: float = 0.
    warmup_ratio: float = 0.03
    lr_scheduler_type: str = "cosine"
    optim: str = "paged_adamw_8bit"
    max_grad_norm: float = 1.0

    # LoRA 配置
    lora_rank: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.05

    # Q-LoRA 和精度配置
    bits: int = 4
    bf16: bool = True # 如果GPU支持，bf16优于fp16

    # 其他
    logging_steps: int = 10
    save_steps: int = 100
    max_seq_length: int = 1024
    report_to: str = "tensorboard"

# 实例化训练配置
sft_train_cfg = SFTConfig()


#### 3.1.2 执行 SFT 训练

这部分整合了模型的加载、LoRA 的配置以及 SFTTrainer 的初始化和启动。

In [ ]:
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig as TrlSFTConfig
from datasets import load_dataset

# 1. 配置4位量化 (Q-LoRA)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# 2. 加载模型和处理器
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    sft_train_cfg.model_id,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(sft_train_cfg.model_id, trust_remote_code=True)
if processor.tokenizer.pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token

# 3. 加载数据集
raw_datasets = load_dataset("json", data_files=sft_train_cfg.data_path, split="train")

# 4. 配置并应用 LoRA
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=sft_train_cfg.lora_rank,
    lora_alpha=sft_train_cfg.lora_alpha,
    lora_dropout=sft_train_cfg.lora_dropout,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 5. 配置 SFTTrainer
training_args = TrlSFTConfig(
    output_dir=sft_train_cfg.output_dir,
    num_train_epochs=sft_train_cfg.num_train_epochs,
    per_device_train_batch_size=sft_train_cfg.per_device_train_batch_size,
    gradient_accumulation_steps=sft_train_cfg.gradient_accumulation_steps,
    gradient_checkpointing=sft_train_cfg.gradient_checkpointing,
    optim=sft_train_cfg.optim,
    logging_steps=sft_train_cfg.logging_steps,
    learning_rate=sft_train_cfg.learning_rate,
    bf16=sft_train_cfg.bf16,
    max_grad_norm=sft_train_cfg.max_grad_norm,
    warmup_ratio=sft_train_cfg.warmup_ratio,
    lr_scheduler_type=sft_train_cfg.lr_scheduler_type,
    save_steps=sft_train_cfg.save_steps,
    max_seq_length=sft_train_cfg.max_seq_length,
    dataset_text_field="conversations",
    dataset_kwargs={"image_column": "image"},
    report_to=sft_train_cfg.report_to
)

trainer = SFTTrainer(
    model=model,
    train_dataset=raw_datasets,
    peft_config=peft_config,
    args=training_args,
    processor=processor
)

# 6. 启动训练
print("--- 开始SFT微调 ---")
trainer.train()
print("--- SFT微调完成！ ---")

# 7. 保存适配器
trainer.save_model(sft_train_cfg.output_dir)
print(f"SFT LoRA 适配器已保存至: {sft_train_cfg.output_dir}")

### 3.2 方法二：使用 LLaMA-Factory 进行微调
LLaMA-Factory 是一个集成了多种高效微调策略的开源框架。它通过命令行工具 llamafactory-cli 提供了强大的功能，非常适合进行系统化的实验。

#### 3.2.1 安装 LLaMA-Factory
首先，我们需要从 GitHub 克隆仓库并进行安装。

In [ ]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!pip install -e LLaMA-Factory

#### 3.2.2 执行 SFT 训练
准备好数据配置文件后，我们可以用一个 Shell 脚本来启动训练。这种方式的好处是所有的超参数都一目了然，便于修改和记录。

```bash
#!/bin/bash

# --- LLaMA-Factory SFT 训练脚本 ---

# 模型和数据路径
MODEL_ID="Qwen/Qwen2.5-VL-7B-Instruct"
# 这是我们在 2.4 节中 dataset_info.json 里定义的名字
DATASET_NAME="qwen_vl_detection"
# 包含 dataset_info.json 和 annotations.jsonl 的目录
DATA_DIR="qwen_vl_dataset"
# 训练输出目录
OUTPUT_DIR="./qwen2.5-vl-sft-lora-llamafactory"

# 启动训练
llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path $MODEL_ID \
    --dataset $DATASET_NAME \
    --dataset_dir $DATA_DIR \
    --template qwen \
    --finetuning_type lora \
    --lora_target q_proj v_proj \
    --output_dir $OUTPUT_DIR \
    --overwrite_cache \
    --overwrite_output_dir \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 100 \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --plot_loss \
    --bf16 \
    --max_seq_length 1024

## 第四部分：推理与部署

经过 SFT 微调后，我们的模型已经准备好了。最后一步是合并 LoRA 权重并进行最终的推理验证。

### 4.1 合并 SFT 权重

这个步骤将 LoRA 适配器的权重融入基础模型，生成一个独立的、可直接部署的完整模型。请注意：无论你使用 TRL 还是 LLaMA-Factory，合并权重的代码都是一样的，只需要修改 `sft_train_cfg.output_dir` 为你实际的 LoRA 权重输出目录即可。

In [ ]:
from peft import PeftModel
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration

# --- 配置 ---
# 根据你使用的训练方法，选择对应的LoRA输出目录
# LORA_ADAPTER_PATH = sft_train_cfg.output_dir # 使用 TRL 训练的
LORA_ADAPTER_PATH = "./qwen2.5-vl-sft-lora-llamafactory" # 使用 LLaMA-Factory 训练的

MERGED_MODEL_PATH = "./qwen2.5-vl-sft-merged"

# --- 加载基础模型 ---
base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    sft_train_cfg.model_id, # model_id 是一样的
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    trust_remote_code=True
)

# --- 加载SFT适配器并合并 ---
final_model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_PATH)
merged_model = final_model.merge_and_unload()

# --- 保存最终模型 ---
merged_model.save_pretrained(MERGED_MODEL_PATH)
processor.save_pretrained(MERGED_MODEL_PATH)
print(f"最终合并后的模型已保存至: {MERGED_MODEL_PATH}")

### 4.2 使用最终模型进行推理
现在，我们可以用最终合并好的模型来进行推理，看看效果。

In [ ]:
# --- 加载最终合并后的模型 ---
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MERGED_MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(MERGED_MODEL_PATH, trust_remote_code=True)

# --- 准备推理输入 ---
with open(data_cfg.ANNOTATION_FILE, 'r', encoding='utf-8') as f:
    test_line = f.readlines()[0]
    test_data = json.loads(test_line)
    
image_path = test_data['image']
image = Image.open(image_path)

prompt = "<image>\n这张图片里有什么？请给出它们的边界框。"
messages = [{"role": "user", "content": prompt}]
text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = processor(text=text_input, images=image, return_tensors="pt").to(model.device)

# --- 执行推理 ---
generated_ids = model.generate(**inputs, max_new_tokens=1024)
response_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

print("--- 最终模型输出 ---")
print(response_text)